In [6]:
%pylab
from numba import (
    jit,
    vectorize,
    float32,
    float64,
    cfunc,
    njit,
    prange,
    get_num_threads,
    set_num_threads,
)
import numpy as np
from math import sqrt
from scipy.special import comb
from scipy.interpolate import interp2d, RectBivariateSpline
from numba import cuda
from numpy import float64, float32, int32, ndarray

cuda.detect()


# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16


@cuda.jit("void(float32[:,:],float32[:])", fastmath=True)
def bruteforce_potential(x, phi):
    i, j = cuda.grid(2)
    if i < x.shape[0] and j < x.shape[0] and j < i:
        r = (
            (x[i, 0] - x[j, 0]) * (x[i, 0] - x[j, 0])
            + (x[i, 1] - x[j, 1]) * (x[i, 1] - x[j, 1])
            + (x[i, 2] - x[j, 2]) * (x[i, 2] - x[j, 2])
        )
        dphi = -1 / sqrt(r)

        cuda.atomic.add(phi, i, dphi)
        cuda.atomic.add(phi, j, dphi)


Np = 4096 * 32

x = np.float32(np.random.rand(Np, 3))
phi = np.zeros(Np)  # cuda.device_array(Np, dtype=np.float32); phi[:] = 0.


threadsperblock = (16, 16)
blockspergrid = (Np // threadsperblock[0], Np // threadsperblock[1])  # int(ceil(Np // threadsperblock))

%timeit bruteforce_potential[blockspergrid,threadsperblock](x,phi)

# @cuda.jit("void(float32[:], float32[:,:], float32[:], float32[:,:], float32)",fastmath=True)
# def GridSurfaceDensity_core_cuda(f, x2d, h, grid, size):
#     res = np.int32(grid.shape[0])
#     dx = np.float32(size / (res - 1))

#     # numba provides this function for working out which element you're
#     # supposed to be accessing
#     i = cuda.grid(1)
#     if i<f.shape[0]: # and j<n3.shape[1]: # check we're in range
#        # print(i)
#         # do work on a single element
#         xs = x2d[i]
#         hs = h[i]
#         hs_sqr = hs*hs
#         hinv = 1 / hs
#         mh2 = f[i] * hinv * hinv * 1.8189136353359467

#         gxmin = max(int((xs[0] - hs) / dx + 1), 0)
#         gxmax = min(int((xs[0] + hs) / dx), res - 1)
#         gymin = max(int((xs[1] - hs) / dx + 1), 0)
#         gymax = min(int((xs[1] + hs) / dx), res - 1)

#         for gx in range(gxmin, gxmax + 1):
#             delta_x_Sqr = xs[0] - gx * dx
#             delta_x_Sqr *= delta_x_Sqr
#             for gy in range(gymin, gymax + 1):
#                 delta_y_Sqr = xs[1] - gy * dx
#                 delta_y_Sqr *= delta_y_Sqr
#                 r = delta_x_Sqr + delta_y_Sqr
#                 if r > hs_sqr:
#                     continue
#                 r = sqrt(r)
#                 q = r * hinv
#                 if q <= 0.5:
#                     kernel = 1 - 6 * q * q * (1 - q)
#                 else: # q <= 1.0:
#                     a = 1 - q
#                     kernel = 2 * a * a * a
#                 cuda.atomic.add(grid, (gx,gy), kernel * mh2)
# cuda.syncthreads()

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Found 1 CUDA devices
id 0         b'NVIDIA T1000'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 0
                              PCI Bus ID: 101
                                    UUID: GPU-b303fbe2-bd8d-69ed-9a8c-01198eed12ed
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


/home/mgrudic/.local/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['sqrt', 'vectorize']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +
/home/mgrudic/.local/lib/python3.9/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


3.88 s ± 18.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
from pytreegrav.bruteforce import Potential_bruteforce_parallel

%timeit  Potential_bruteforce_parallel(x,np.ones(Np),np.zeros(Np))

4.55 s ± 87.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
phi.copy_to_host()

array([-1883.2377, -1630.0737, -2068.1045, ..., -1790.0035, -1583.7726,
       -1866.4478], dtype=float32)

In [ ]:
from pytreegrav.bruteforce import Po